In [1]:
# Data Analysis
   
import pandas as pd
import numpy as np
import os 
import missingno as msno
import matplotlib.pyplot as plt

import math
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

    
# Data View
pd.options.display.max_columns = 200

# Import Basic Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

os.chdir('C:\\Users\\jason\\OneDrive\\Desktop\\NCU\\8535_Univariate Analysis\\Week 6')

In [2]:
df_raw= pd.read_csv(r"C:\Users\jason\OneDrive\Desktop\NCU\8535_Univariate Analysis\Week 6\real_estate_inventory.csv")

In [3]:
#create new variable for with month
from datetime import datetime as dt
df_raw["date"] = pd.to_datetime(df_raw['date']).dt.to_period('D')
df_raw['month'] = df_raw["date"].dt.month

In [4]:
#rename the levels in 'type' for townhome, condo single family
type = {'townhome': 0,'condo': 1, 'single family':2}
df_raw.type = [type[item] for item in df_raw.type]
#rename the levels in 'state' for NY, NJ, PA
state = {'NY': 0,'NJ': 1, 'PA':2}
df_raw.state = [state[item] for item in df_raw.state]

In [5]:
#Normalize using Min Max Transform for price, squareft, and acreage

from sklearn import preprocessing
normvar = df_raw[["price", "squareft", "acreage"]]
d = preprocessing.normalize(normvar,axis=0)
scaled_df = pd.DataFrame(d, columns=["price_n", "squareft_n", "acreage_n"])
df_norm = pd.concat([df_raw, scaled_df], axis=1)

In [6]:
#Creating training and testing data
X = df_norm.drop(['price', 'squareft', 'acreage', 'purchased', 'date'], axis=1)
y = df_norm['purchased']

from sklearn.model_selection import train_test_split
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

In [68]:
from sklearn import svm
#create a classifier w/linear kernel
cls = svm.SVC(kernel="linear")
#train the model
cls.fit(X_train,y_train)
#predict the response
predl = cls.predict(X_test)

In [107]:
svm = svm.SVC(kernel="linear")
svm.fit(X_train,y_train)
svm_predict = svm.predict(X_test)
svm_cv_score = cross_val_score(svm, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, svm_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, svm_predict))
print('\n')
print("=== All AUC Scores ===")
print(svm_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Support Vector Machine: ", svm_predict.mean())

C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (

=== Confusion Matrix ===
[[489 312]
 [517 332]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.49      0.61      0.54       801
           1       0.52      0.39      0.44       849

    accuracy                           0.50      1650
   macro avg       0.50      0.50      0.49      1650
weighted avg       0.50      0.50      0.49      1650



=== All AUC Scores ===
[nan nan nan nan nan nan nan nan nan nan]


=== Mean AUC Score ===
Mean AUC Score - Naive Bayes:  0.3903030303030303


C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(


In [81]:
from sklearn import metrics
#accuracy
print("acuracy:", metrics.accuracy_score(y_test,predl))
#precision score
print("precision:", metrics.precision_score(y_test,predl))
#recall score
print("recall" , metrics.recall_score(y_test,predl))
print(metrics.classification_report(y_test, predl))

acuracy: 0.49757575757575756
precision: 0.515527950310559
recall 0.3910482921083628
              precision    recall  f1-score   support

           0       0.49      0.61      0.54       801
           1       0.52      0.39      0.44       849

    accuracy                           0.50      1650
   macro avg       0.50      0.50      0.49      1650
weighted avg       0.50      0.50      0.49      1650



In [101]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

C_range = np.logspace(1, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.333, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'C': 10.0, 'gamma': 0.001} with a score of 0.99


In [104]:
from sklearn import svm
#create a classifier with tuning gamma and C
clf = svm.SVC(gamma=0.001, C=10)
#train the model
clf.fit(X_train,y_train)
#predict the response
pred = clf.predict(X_test)

In [105]:
from sklearn import metrics
#accuracy
print("acuracy:", metrics.accuracy_score(y_test,pred))
#precision score
print("precision:", metrics.precision_score(y_test,pred))
#recall score
print("recall" , metrics.recall_score(y_test,pred))
print(metrics.classification_report(y_test, pred))

acuracy: 0.49393939393939396
precision: 0.5135135135135135
recall 0.3133097762073027
              precision    recall  f1-score   support

           0       0.48      0.69      0.57       801
           1       0.51      0.31      0.39       849

    accuracy                           0.49      1650
   macro avg       0.50      0.50      0.48      1650
weighted avg       0.50      0.49      0.48      1650



In [109]:
#Run Support Vector Machine with tuned hyperparmeters

clf_cv_score = cross_val_score(clf, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, pred))
print('\n')
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Support Vector Machine: ", clf_cv_score.mean())

C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "C:\Users\jason\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\Users\jason\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (

=== Confusion Matrix ===
[[549 252]
 [583 266]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.48      0.69      0.57       801
           1       0.51      0.31      0.39       849

    accuracy                           0.49      1650
   macro avg       0.50      0.50      0.48      1650
weighted avg       0.50      0.49      0.48      1650



=== All AUC Scores ===
[nan nan nan nan nan nan nan nan nan nan]


=== Mean AUC Score ===
Mean AUC Score - Support Vector Machine:  nan
